In [4]:
!pip install crewai

In [8]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 58.6 MB/s eta 0:00:00:00:0100:01


In [9]:
# Sentence Transformers for embeddings
!pip install -q sentence-transformers

# If you want to use LangChain later (e.g., for RAG pipeline or embeddings)
!pip install -q langchain faiss-cpu

# Optional: Hugging Face for LLMs (DeepSeek, etc.)
!pip install -q transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have 

## we create tasks

In [5]:
from textwrap import dedent 
from crewai import Task

### Build FAISS Vector Store from .csv

In [19]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import pickle

# Load your CSV
df = pd.read_csv("/kaggle/input/sikka-dataset/Sikka_APIs - Sikka_APIs.csv")
df.fillna("", inplace=True)

# Create a formatted chunk with all key details
docs = []
metadata = []

for _, row in df.iterrows():
    chunk = f"""API Name: {row['API Name']}
Description: {row['Description']}
Endpoint: {row['API Endpoints']}
Documentation Link: {row['Document Link']}"""
    
    docs.append(chunk)
    metadata.append({
        "api_name": row["API Name"],
        "endpoint": row["API Endpoints"],
        "doc_link": row["Document Link"]
    })

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(docs, show_progress_bar=True)

# Build FAISS index
dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Save to disk
faiss.write_index(index, "sikka_faiss.index")
with open("sikka_docs.pkl", "wb") as f:
    pickle.dump(docs, f)
with open("sikka_meta.pkl", "wb") as f:
    pickle.dump(metadata, f)


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
def sikka_api_retriever(query, top_k=3):
    import pickle
    import faiss
    from sentence_transformers import SentenceTransformer

    # Load FAISS and data
    index = faiss.read_index("sikka_faiss.index")
    with open("sikka_docs.pkl", "rb") as f:
        docs = pickle.load(f)
    with open("sikka_meta.pkl", "rb") as f:
        metadata = pickle.load(f)

    model = SentenceTransformer("all-MiniLM-L6-v2")
    
    query_embedding = model.encode([query])

    D, I = index.search(query_embedding, top_k)
    results = []
    for i in I[0]:
        result = {
            "doc": docs[i],
            "api_name": metadata[i]["api_name"],
            "endpoint": metadata[i]["endpoint"],
            "doc_link": metadata[i]["doc_link"]
        }
        results.append(result)
    return results


In [23]:
results = sikka_api_retriever("get patient outstanding balances and support payment plans")
for r in results:
    print(f"API Name: {r['api_name']}")
    print(f"Endpoint: {r['endpoint']}")
    print(f"Doc: {r['doc']}")
    print(f"Doc Link: {r['doc_link']}\n")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

API Name: pending_procedures
Endpoint: https://api.sikkasoft.com/v4/pending_procedures
Doc: API Name: pending_procedures
Description: Returns pending procedures for the patient with available balance data from practice
Endpoint: https://api.sikkasoft.com/v4/pending_procedures
Documentation Link: https://apidocs.sikkasoft.com/#9086881d-f5cc-4435-93fe-1ed8dec926ce
Doc Link: https://apidocs.sikkasoft.com/#9086881d-f5cc-4435-93fe-1ed8dec926ce

API Name: patient_balance
Endpoint: https://api.sikkasoft.com/v4/patient_balance
Doc: API Name: patient_balance
Description: Return patient balance from practice management systems.
Endpoint: https://api.sikkasoft.com/v4/patient_balance
Documentation Link: https://apidocs.sikkasoft.com/#f24e3b18-ddee-48c5-a44e-79dbc281ffa7
Doc Link: https://apidocs.sikkasoft.com/#f24e3b18-ddee-48c5-a44e-79dbc281ffa7

API Name: suspended_credit_transactions
Endpoint: https://api.sikkasoft.com/v4/suspended_credit_transactions
Doc: API Name: suspended_credit_transaction

In [24]:
results = sikka_api_retriever(  "get patient balances, due amounts, and setup options for payment plans, "
  "installment plans, recurring payments, or financial assistance.")
for r in results:
    print(f"API Name: {r['api_name']}")
    print(f"Endpoint: {r['endpoint']}")
    print(f"Doc: {r['doc']}")
    print(f"Doc Link: {r['doc_link']}\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

API Name: payment_plans
Endpoint: https://api.sikkasoft.com/v4/payment_plans
Doc: API Name: payment_plans
Description: Returns payment plans data from practice if practice management system storing payment plans
Endpoint: https://api.sikkasoft.com/v4/payment_plans
Documentation Link: https://apidocs.sikkasoft.com/#53518438-54e5-4769-ba41-f7308a5f6bf7
Doc Link: https://apidocs.sikkasoft.com/#53518438-54e5-4769-ba41-f7308a5f6bf7

API Name: patient_balance
Endpoint: https://api.sikkasoft.com/v4/patient_balance
Doc: API Name: patient_balance
Description: Return patient balance from practice management systems.
Endpoint: https://api.sikkasoft.com/v4/patient_balance
Documentation Link: https://apidocs.sikkasoft.com/#f24e3b18-ddee-48c5-a44e-79dbc281ffa7
Doc Link: https://apidocs.sikkasoft.com/#f24e3b18-ddee-48c5-a44e-79dbc281ffa7

API Name: pending_procedures
Endpoint: https://api.sikkasoft.com/v4/pending_procedures
Doc: API Name: pending_procedures
Description: Returns pending procedures for

In [6]:
from textwrap import dedent
from crewai import Task

class PaymentSystemTasks():
    
    def researcher_task(self, agent, system_goal, reference_solution):
        return Task(
            description=dedent(f"""\
                Analyze the reference solution "{reference_solution}" and extract key business and technical requirements 
                for building a {system_goal}.

                Focus on identifying:
                - Payment flow types (e.g., credit, debit, recurring plans)
                - Compliance needs (e.g., HIPAA, PCI)
                - Features like surcharges, payment links, terminals, etc.
                - Integration requirements with healthcare systems

                This research will guide downstream agents in selecting APIs and generating code.
            """),
            expected_output=dedent(f"""\
                A bullet-point summary of core features, compliance requirements, 
                and integration expectations based on {reference_solution}, 
                which will be used as a blueprint for implementing the {system_goal}.
            """),
            agent=agent,
            async_execution=False
        )

    def retriever_task(self, agent, extracted_requirements):
        return Task(
            description=dedent(f"""\
                Based on the following system requirements:

                {extracted_requirements}

                Step 1: Use the FAISS-powered Sikka API retriever tool to identify the top relevant API endpoints 
                that can fulfill the specified needs.

                Step 2: For each matched API, visit its documentation link (from the Sikka CSV) and extract the list of:
                - Required parameters
                - Optional parameters

                These parameters will be used by the Developer Agent to generate accurate backend/frontend code.
            """),
            expected_output=dedent(f"""\
                A structured list of 2–4 Sikka API endpoints, where each entry includes:
                - API Name
                - Endpoint path
                - One-line summary (from description)
                - Documentation link
                - Required parameters (list of strings)
                - Optional parameters (list of strings)
            """),
            agent=agent,
            async_execution=False
        )
        def developer_task(self, agent, system_goal, reference_solution, extracted_features, api_details):
        return Task(
            description=dedent(f"""\
                Using the design blueprint and APIs gathered in previous steps, build a complete full-stack {system_goal} 
                that replicates the core functionality of "{reference_solution}" and incorporates the features outlined below.

                --- System Features to Implement ---
                {extracted_features}

                --- API Endpoints to Integrate ---
                The following endpoints must be integrated into the backend logic, using the specified parameters:

                {api_details}

                --- Your Responsibilities ---
                1. Write a functional backend (e.g., Flask or FastAPI) that handles:
                   - API requests
                   - Input validation for required parameters
                   - Integration with the relevant Sikka API endpoints
                   - Proper handling of request/response data

                2. Write a frontend (HTML or React) that:
                   - Provides clean UI for users to input required fields
                   - Displays output from Sikka APIs (e.g., patient balances, payment options)
                   - Handles user errors or missing inputs gracefully

                3. Ensure all frontend inputs map correctly to backend logic and API parameters.

                4. Include any assumptions, mocked tokens, or dummy practice/patient IDs needed for simulation.

                5. The generated code should be clean, working, and runnable with minimal modification.

                Your response should include:
                - The full backend code
                - The full frontend code
                - Comments that indicate where each API is used and how it maps to inputs/outputs
            """),
            expected_output=dedent(f"""\
                A complete, runnable full-stack codebase for the {system_goal}.

                Output should include:
                - Backend code (Flask or FastAPI) with integrated API logic
                - Frontend code (HTML or React) with inputs and display logic
                - API integration points with correct use of parameters
                - Any helper functions, data structures, or mocks needed to simulate usage
            """),
            agent=agent,
            async_execution=False
        )
        def reviewer_task(self, agent, full_stack_code, extracted_features, api_details):
        return Task(
            description=dedent(f"""\
                You are acting as a senior software reviewer. Your role is to critically review the following 
                full-stack implementation of a {len(api_details)}-API integrated Payment System and ensure 
                it is functionally correct, complete, and error-free.

                --- Key Responsibilities ---

                1. **Code Correctness**:
                   - Carefully analyze the backend and frontend code for logical correctness.
                   - Identify bugs, edge cases, or missing error handling.
                   - Ensure the API integration sections work as written and simulate expected behavior.

                2. **Feature Completeness**:
                   - Validate that all features identified by the Research Agent have been implemented.
                   - Features include:
                     {extracted_features}

                3. **API Usage Accuracy**:
                   - Confirm that all Sikka APIs identified by the Retriever Agent are used appropriately.
                   - Match each API to the correct feature it's intended to support.
                   - APIs included:
                     {api_details}

                4. **Parameter Validation**:
                   - Cross-check each API's usage to verify required/optional parameters are:
                     - Passed from the frontend to the backend correctly
                     - Integrated in accordance with the Sikka API docs
                   - Make note of any missing or incorrectly mapped parameters.

                5. **Overall Code Quality**:
                   - Evaluate readability, structure, and modularity.
                   - Suggest improvements in design, error handling, or best practices.

                --- Deliverables ---

                Your review should contain:
                - A section-by-section critique (backend and frontend)
                - A checklist matching each API to its associated feature
                - A validation of each parameter mapping
                - A list of improvements or refactors
                - Final verdict: ✅ Acceptable / ❌ Needs Fixes, with reasoning
            """),
            expected_output=dedent(f"""\
                A detailed review report with:
                - Bug reports and logic issues (if any)
                - Feature vs implementation checklist
                - Parameter mapping validation
                - Suggestions for improvements
                - Final decision: ✅ Approve or ❌ Request changes
            """),
            agent=agent,
            async_execution=False
        )


In [25]:
!pip install exa_py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.2
    Uninstalling httpx-0.27.2:
      Successfully uninstalled httpx-0.27.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
litellm 1.60.2 requires httpx<0.28.0,>=0.23.0, but you have httpx 0.28.1 which is incompatible.


In [29]:
!pip install exa-py

In [39]:
from crewai import Agent
from tools import ExaSearchToolset

class PaymentSystemAgents:
    def __init__(self, faiss_tool):
        self.faiss_tool = faiss_tool

    def research_agent(self):
        return Agent(
            role="Research Specialist",
            goal="Extract key business and technical requirements from the reference payment system (e.g., Nadapayments) to guide full-stack application design.",
            tools=ExaSearchToolset.tools(),
            backstory=dedent("""\
                A domain researcher with expertise in fintech and healthcare payment platforms. 
                You are tasked with analyzing real-world systems like Nadapayments to identify must-have features 
                such as surcharges, recurring payments, compliance requirements (HIPAA/PCI), and integration expectations 
                in medical practice software. Your output will drive downstream API selection and code generation.
            """),
            verbose=True
        )

    def retriever_agent(self):
        return Agent(
            role="Sikka API Retriever",
            goal="Identify the most relevant Sikka API endpoints for each requested feature and extract their required and optional parameters from documentation links.",
            tools=[
                self.faiss_tool,
                *ExaSearchToolset.tools()
            ],
            backstory=dedent("""\
                You are an expert in API retrieval and integration. Your role is to find the most relevant Sikka API endpoints 
                that match the system requirements using semantic vector search, and for each matched endpoint, 
                fetch its required and optional parameters from the linked API documentation using Exa web search. 
                This information will be used by the Developer Agent to write functional full-stack code.
            """),
            verbose=True
        )

    def developer_agent(self):
        return Agent(
            role="Full-Stack Payment System Developer",
            goal="Write clean, functional backend and frontend code to implement a payment system inspired by a reference solution using the provided APIs and parameters.",
            backstory=dedent("""\
                You are a skilled full-stack software engineer with expertise in developing healthcare-related applications. 
                You will receive the required features extracted from a real-world solution (e.g., Nadapayments) 
                and a set of identified Sikka APIs with their parameters. 
                Your job is to generate both backend and frontend code to build a working payment system.

                You are expected to:
                - Use the identified API endpoints from the Retriever Agent appropriately
                - Handle all required and optional parameters accurately in both frontend and backend
                - Create API calls in the backend and link them to frontend forms
                - Ensure correct routing, validation, and error handling
                - Return code that is readable, structured, and runnable
            """),
            verbose=True
        )

    def reviewer_agent(self):
        return Agent(
            role="Senior Code Reviewer",
            goal="Critically evaluate the generated code for correctness, feature coverage, proper API usage, and parameter mapping based on documentation.",
            backstory=dedent("""\
                You are a senior software engineer and quality assurance expert with deep experience in reviewing 
                full-stack healthcare and fintech systems. You’ve been asked to review a payment system implementation 
                created by another developer agent.

                You must ensure:
                - The code is free from logic errors or implementation flaws
                - All the requested features from the Research Agent are implemented
                - Every identified Sikka API is used appropriately
                - Parameters used in the code match exactly with the required and optional parameters 
                  listed in the documentation links provided by the Retriever Agent
                - The frontend is correctly wired to the backend, and inputs map to the correct parameters
                - The backend makes correct API calls with validated input

                You will return a detailed review report, including a final recommendation to approve or request changes.
            """),
            verbose=True
        )


ModuleNotFoundError: No module named 'tools'

In [34]:
!pip install crewai-tools

  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.3/573.3 kB 12.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 55.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 7.7 MB/s eta 0:00:00
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:

In [ ]:
import os
from langchain.tools import Tool
from exa_py import Exa

os.environ["EXA_API_KEY"] = "your_exa_api_key"
exa_client = Exa(api_key=os.environ["EXA_API_KEY"])

def sikka_api_retriever(query):
    # implement your real retriever logic
    return "Mocked result for now"

faiss_retriever_tool = Tool(
    name="Sikka API Retriever",
    description="Retrieve relevant Sikka API endpoints using FAISS-based semantic search.",
    func=sikka_api_retriever
)

def extract_parameters_from_doc_link(doc_link: str) -> str:
    query = f"required and optional parameters for Sikka API documented at {doc_link}"
    results = exa_client.search_and_contents(query, use_autoprompt=True, num_results=3)
    return "\n\n".join([f"{r.title}\n{r.url}\n{r.text}" for r in results.results])

exa_doc_tool = Tool(
    name="Exa API Parameter Extractor",
    description="Extracts required/optional parameters from Swagger doc links using Exa.ai.",
    func=extract_parameters_from_doc_link
)

class PaymentSystemTools:
    @staticmethod
    def get_tools():
        return {"faiss": faiss_retriever_tool, "exa": exa_doc_tool}

    @staticmethod
    def list_for_retriever():
        return [faiss_retriever_tool, exa_doc_tool]

    @staticmethod
    def list_for_researcher():
        return [exa_doc_tool]
